In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

warnings.filterwarnings('ignore')

In [2]:
# --- 1. 설정 및 데이터 로드 ---
DATASET_DIRECTORY = 'CICIoT2023/'
# 튜닝 속도를 위해 샘플링 비율을 낮게 설정 (5%만 사용)
# 실제 학습때는 이 비율을 높이거나 찾은 파라미터를 적용하세요.
TUNING_SAMPLE_RATIO = 0.05

# 컬럼 정의 (기존 코드 참조)
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

# 8-Class 매핑 (대표적인 시나리오로 튜닝 진행)
dict_8_classes = {
    'DDoS-RSTFINFlood': 'DDoS', 'DDoS-PSHACK_Flood': 'DDoS', 'DDoS-SYN_Flood': 'DDoS',
    'DDoS-UDP_Flood': 'DDoS', 'DDoS-TCP_Flood': 'DDoS', 'DDoS-ICMP_Flood': 'DDoS',
    'DDoS-SynonymousIP_Flood': 'DDoS', 'DDoS-ACK_Fragmentation': 'DDoS',
    'DDoS-UDP_Fragmentation': 'DDoS', 'DDoS-ICMP_Fragmentation': 'DDoS',
    'DDoS-SlowLoris': 'DDoS', 'DDoS-HTTP_Flood': 'DDoS', 'DoS-UDP_Flood': 'DoS',
    'DoS-SYN_Flood': 'DoS', 'DoS-TCP_Flood': 'DoS', 'DoS-HTTP_Flood': 'DoS',
    'Mirai-greeth_flood': 'Mirai', 'Mirai-greip_flood': 'Mirai', 'Mirai-udpplain': 'Mirai',
    'Recon-PingSweep': 'Recon', 'Recon-OSScan': 'Recon', 'Recon-PortScan': 'Recon',
    'VulnerabilityScan': 'Recon', 'Recon-HostDiscovery': 'Recon',
    'DNS_Spoofing': 'Spoofing', 'MITM-ArpSpoofing': 'Spoofing',
    'BenignTraffic': 'Benign', 'BrowserHijacking': 'Web', 'Backdoor_Malware': 'Web',
    'XSS': 'Web', 'Uploading_Attack': 'Web', 'SqlInjection': 'Web',
    'CommandInjection': 'Web', 'DictionaryBruteForce': 'BruteForce'
}

def load_subset_data(directory, sample_ratio):
    """튜닝을 위해 데이터의 일부만 로드"""
    all_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    # 파일 개수도 줄여서 로딩 (속도 최적화)
    subset_files = all_files[:10]

    dfs = []
    print(f"데이터 로드 중 (파일 {len(subset_files)}개, 비율 {sample_ratio})...")
    for f in subset_files:
        try:
            path = os.path.join(directory, f)
            df = pd.read_csv(path)
            # 샘플링
            df_sample = df.sample(frac=sample_ratio, random_state=42)
            dfs.append(df_sample)
        except Exception as e:
            continue

    full_df = pd.concat(dfs, ignore_index=True)

    # 전처리
    full_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    full_df.fillna(0, inplace=True)

    X = full_df[X_columns]
    y = full_df[y_column].map(dict_8_classes).fillna('Benign')

    return X, y

# 데이터 로드
if os.path.exists(DATASET_DIRECTORY):
    X, y = load_subset_data(DATASET_DIRECTORY, TUNING_SAMPLE_RATIO)

    # 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 레이블 인코딩
    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    # 훈련/검증 세트 분리
    X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_enc, test_size=0.2, random_state=42)

    print(f"학습 데이터 크기: {X_train.shape}")

    # --- 2. LightGBM 하이퍼파라미터 튜닝 ---
    print("\n[LightGBM] 하이퍼파라미터 튜닝 시작...")

    # LightGBM 파라미터 범위 설정
    # num_leaves: 트리의 복잡도 결정 (가장 중요). 2^max_depth보다 작게 설정.
    # min_child_samples: 과적합 방지.
    lgbm_params = {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [31, 64, 128],
        'max_depth': [-1, 10, 20],
        'min_child_samples': [20, 50, 100],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    lgbm = LGBMClassifier(random_state=42, verbose=-1)

    # RandomizedSearchCV 실행 (n_iter=10: 10번의 랜덤 조합 시도)
    lgbm_search = RandomizedSearchCV(
        lgbm, lgbm_params, n_iter=10, scoring='accuracy', cv=3, verbose=1, random_state=42, n_jobs=-1
    )
    lgbm_search.fit(X_train, y_train)

    print(f"LightGBM 최적 파라미터: {lgbm_search.best_params_}")
    print(f"LightGBM 최고 정확도 (CV): {lgbm_search.best_score_:.4f}")


    # --- 3. XGBoost 하이퍼파라미터 튜닝 ---
    print("\n[XGBoost] 하이퍼파라미터 튜닝 시작...")

    # XGBoost 파라미터 범위 설정
    xgb_params = {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 6, 10],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }

    xgb_model = XGBClassifier(random_state=42, eval_metric='mlogloss')

    xgb_search = RandomizedSearchCV(
        xgb_model, xgb_params, n_iter=5, scoring='accuracy', cv=3, verbose=1, random_state=42, n_jobs=-1
    )
    xgb_search.fit(X_train, y_train)

    print(f"XGBoost 최적 파라미터: {xgb_search.best_params_}")
    print(f"XGBoost 최고 정확도 (CV): {xgb_search.best_score_:.4f}")

else:
    print("데이터 디렉토리를 찾을 수 없어 튜닝을 수행하지 못했습니다.")

데이터 로드 중 (파일 10개, 비율 0.05)...
학습 데이터 크기: (94678, 46)

[LightGBM] 하이퍼파라미터 튜닝 시작...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
LightGBM 최적 파라미터: {'subsample': 0.8, 'num_leaves': 128, 'n_estimators': 500, 'min_child_samples': 20, 'max_depth': 10, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
LightGBM 최고 정확도 (CV): 0.9928

[XGBoost] 하이퍼파라미터 튜닝 시작...
Fitting 3 folds for each of 5 candidates, totalling 15 fits
XGBoost 최적 파라미터: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.2, 'colsample_bytree': 0.8}
XGBoost 최고 정확도 (CV): 0.9932
